# Proyecto de Tesis

In [1]:
import torch
import torchvision

import matplotlib.pyplot as plt

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
dataloader = {
    'train': torch.utils.data.DataLoader(torchvision.datasets.MNIST('./Convolucional', train=True, download=True,
                       transform = torchvision.transforms.Compose([
                            torchvision.transforms.ToTensor(),
                            torchvision.transforms.Normalize((0.1307,), (0.3081,))
                            ])
                      ), batch_size=64, shuffle=True, pin_memory=True),
    'test': torch.utils.data.DataLoader(torchvision.datasets.MNIST('./Convolucional', train=False, download=True,
                     transform = torchvision.transforms.Compose([
                        torchvision.transforms.ToTensor(),
                        torchvision.transforms.Normalize((0.1307,), (0.3081,))
                        ])
                     ), batch_size=64, shuffle=False, pin_memory=True)
}

9913344it [00:03, 3198812.72it/s]                             


Extracting ./Convolucional\MNIST\raw\train-images-idx3-ubyte.gz to ./Convolucional\MNIST\raw



29696it [00:00, 9783524.59it/s]          


Extracting ./Convolucional\MNIST\raw\train-labels-idx1-ubyte.gz to ./Convolucional\MNIST\raw



1649664it [00:00, 2797368.19it/s]                             


Extracting ./Convolucional\MNIST\raw\t10k-images-idx3-ubyte.gz to ./Convolucional\MNIST\raw



5120it [00:00, ?it/s]                   


Extracting ./Convolucional\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./Convolucional\MNIST\raw



In [40]:
def block(c_in, c_out, k=3, p=1, s=1, pk=2, ps=2):
    return torch.nn.Sequential(
        torch.nn.Conv2d(c_in, c_out, k, padding=p, stride=s),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(pk, stride=ps)
    )

class CNN(torch.nn.Module):
  def __init__(self, n_channels=1, n_outputs=10):
    super().__init__()
    self.conv1 = block(n_channels, 64)
    self.conv2 = block(64, 128)
    self.fc = torch.nn.Linear(128*7*7, n_outputs)

  def forward(self, x):
    x = self.conv1(x)
    x = self.conv2(x)
    x = x.view(x.shape[0], -1)
    x = self.fc(x)
    return x

In [41]:
import numpy as np

def fit(model, dataloader, epochs=5):
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterion = torch.nn.CrossEntropyLoss()
    for epoch in range(1, epochs+1):
        model.train()
        convTrainLoss, convTrainAcc = [], []
        batches = dataloader['train']
        for batch in batches:
            X, y = batch
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            y_hat = model(X)
            loss = criterion(y_hat, y)
            loss.backward()
            optimizer.step()
            convTrainLoss.append(loss.item())
            acc = (y == torch.argmax(y_hat, axis=1)).sum().item() / len(y)
            convTrainAcc.append(acc)
        batches = dataloader['test']
        convValLoss, convValAcc = [], []
        model.eval()
        with torch.no_grad():
            for batch in batches:
                X, y = batch
                X, y = X.to(device), y.to(device)
                y_hat = model(X)
                loss = criterion(y_hat, y)
                convValLoss.append(loss.item())
                acc = (y == torch.argmax(y_hat, axis=1)).sum().item() / len(y)
                convValAcc.append(acc)
        print(f"Epoch {epoch}/{epochs} loss {np.mean(convTrainLoss):.5f} val_loss {np.mean(convValLoss):.5f} acc {np.mean(convTrainAcc):.5f} val_acc {np.mean(convValAcc):.5f}")
    return convTrainLoss, convValLoss, convTrainAcc, convValAcc

In [42]:
CNNmodel = CNN()
convTrainLoss, convValLoss, convTrainAcc, convValAcc = fit(CNNmodel, dataloader)

Epoch 1/5 loss 0.11572 val_loss 0.03976 acc 0.96507 val_acc 0.98656
Epoch 2/5 loss 0.04101 val_loss 0.05186 acc 0.98746 val_acc 0.98348
Epoch 3/5 loss 0.02777 val_loss 0.03330 acc 0.99107 val_acc 0.98935
Epoch 4/5 loss 0.02187 val_loss 0.03126 acc 0.99300 val_acc 0.98965
Epoch 5/5 loss 0.01423 val_loss 0.04511 acc 0.99569 val_acc 0.98746


In [2]:
import numpy as np

In [15]:
Z = np.ones((10,10))
Z[2:8,2:8] = 0
Z

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 0., 0., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0., 0., 0., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])

In [13]:
Z[5-2:5+3,6-2:6+3]

array([[1., 1., 1., 1., 0.],
       [1., 1., 1., 1., 0.],
       [1., 1., 1., 1., 0.],
       [1., 1., 1., 1., 0.],
       [1., 1., 1., 1., 0.]])

In [16]:
train = torchvision.datasets.MNIST('./GNN', train=True, download=True)
test = torchvision.datasets.MNIST('./GNN', train=False, download=True)

9913344it [00:03, 2702125.55it/s]                             


Extracting ./GNN\MNIST\raw\train-images-idx3-ubyte.gz to ./GNN\MNIST\raw



29696it [00:00, 3781584.59it/s]          


Extracting ./GNN\MNIST\raw\train-labels-idx1-ubyte.gz to ./GNN\MNIST\raw



1649664it [00:00, 3312643.96it/s]                             


Extracting ./GNN\MNIST\raw\t10k-images-idx3-ubyte.gz to ./GNN\MNIST\raw



5120it [00:00, ?it/s]                   

Extracting ./GNN\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./GNN\MNIST\raw



In [104]:
def createNodesandEdges(data):
    listNodes, listEdges = [], []
    for n, imgit in enumerate(data):
        imgit = imgit.numpy()
        imgit = np.where(imgit == 0, -1, imgit)
        imgit = np.pad(imgit, [(2, 2), (2, 2)], "constant", constant_values = (0))

        cnt = 0

        for i in range(2, 30):
            for j in range(2, 30):
                if (imgit[i][j] > -1):
                    imgit[i][j] = cnt
                    cnt = cnt + 1
    
        edges = []
        nodes = np.zeros((cnt, 2))

        for i in range(2, 30):
            for j in range(2, 30):
                if (imgit[i][j] == -1):
                    continue

                filter = imgit[i - 2:i + 3, j - 2:j + 3].flatten()
                filterVal = filter[[6, 7, 8, 11, 13, 16, 17, 18]]

                nodes[filter[12]][0] = i - 2
                nodes[filter[12]][1] = j - 2

                for val in filterVal:
                    if (not val == -1):
                        edges.append([filter[12], val])
    
        #np.save("./GNN/MNIST/Graphs/"+str(n), edges)
        #np.save("./GNN/MNIST/Nodes/"+str(n), nodes)
        listNodes.append(nodes)
        listEdges.append(edges)
    return listNodes, listEdges

In [105]:
listNodes, listEdges = createNodesandEdges(train.data)

KeyboardInterrupt: 

In [108]:
len(train.targets)

60000